
## About this project

We want a csv with data about most sold products in amazon, because we want to check in another websites if this products are cheaper and we can re-sell in amazon. To do this, we need specific information like the product model,brand,price, merchants, how often is sold, etc.

I will scrap in a specific main category of products (I already know some other sites selling this products with cheaper prices :D ). 

Technologies used:
- Selenium
- BeautifulSoup (utils.py)
- Pandas

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from selenium import webdriver
import pandas as pd
import time
import random
from utils import (soup_content_from_url,
                    elements_from_soup_bestsellers_web, 
                    subcategories_from_bestsellers_web,
                    detail_dict_from_product_page,
                    load_obj,
                    save_obj)

In [4]:
driver = webdriver.Firefox()
soups = []

## Scrapping the main category
The problem is amazon show me just the top 100 most sold products, and is a very tiny market for me. So I will scrap many subcategories in this main category to get more than 100 products. Just we'll use the subcategories list in category web page.

### Set initial Category URL
We will set the main category url. 

In [5]:
base_url = 'https://www.amazon.es'
bestsellers_webs=[
    #{'name':'beauty','url':'https://www.amazon.es/gp/bestsellers/beauty/ref=zg_bs_pg_%i?ie=UTF8&pg=%i'},
    #{'name':'office','url': base_url + '/gp/bestsellers/office/ref=zg_bs_pg_%i?ie=UTF8&pg=%i'},
    {'name':'computers','url': base_url + '/gp/bestsellers/computers/ref=zg_bs_nav_%i?ie=UTF8&pg=%i'},
]

### Getting the html content from the URL
Use the beautiful soup library to get the html content in a useful format. 

In [6]:
def soup_from_bestsellers_url(url,page):
    url = w['url']%(page,page)
    print(url)
    return soup_content_from_url(driver, url), url
     
for w in bestsellers_webs:
    print(f"Category: {w['name']}")
    w['soups'] = []
    
    soup, url = soup_from_bestsellers_url(w['url'],page=1)
    w['soups'].append(soup)
    # Sleep is to avoid banning from amazon servers.
    time.sleep(5)
    
    soup_from_bestsellers_url(w['url'], page=2)
    w['soups'].append(soup)
    time.sleep(5)

Category: computers
https://www.amazon.es/gp/bestsellers/computers/ref=zg_bs_nav_1?ie=UTF8&pg=1
https://www.amazon.es/gp/bestsellers/computers/ref=zg_bs_nav_2?ie=UTF8&pg=2


### Getting subcategories
We use the html soup resource to preprocess and get its subcategories

In [7]:
category = bestsellers_webs[0]
subcategories = subcategories_from_bestsellers_web(category['soups'][0])
## We delete the first element, its an autoreference url
subcategories.pop(0)
subcategories

[{'name': 'Accesorios',
  'url': 'https://www.amazon.es/gp/bestsellers/computers/937753031/ref=zg_bs_nav_computers_1_computers/257-7699657-9620336'},
 {'name': 'Componentes',
  'url': 'https://www.amazon.es/gp/bestsellers/computers/937912031/ref=zg_bs_nav_computers_1_computers/257-7699657-9620336'},
 {'name': 'Dispositivos de red',
  'url': 'https://www.amazon.es/gp/bestsellers/computers/937958031/ref=zg_bs_nav_computers_1_computers/257-7699657-9620336'},
 {'name': 'Monitores',
  'url': 'https://www.amazon.es/gp/bestsellers/computers/937992031/ref=zg_bs_nav_computers_1_computers/257-7699657-9620336'},
 {'name': 'Ordenadores de sobremesa',
  'url': 'https://www.amazon.es/gp/bestsellers/computers/937994031/ref=zg_bs_nav_computers_1_computers/257-7699657-9620336'},
 {'name': 'Portátiles',
  'url': 'https://www.amazon.es/gp/bestsellers/computers/938008031/ref=zg_bs_nav_computers_1_computers/257-7699657-9620336'},
 {'name': 'Servidores',
  'url': 'https://www.amazon.es/gp/bestsellers/comput

## Scrapping subcategories

From now we have a list with each subcategory, and we can use this URLs to get a products list with information like the product name, price, the amazon product page url.

### Getting the HTML content from each subcategory
Initially we need this content in beautfulsoup format in order to could scrap each page looking for the data.

In [8]:
for s in subcategories:
    print(f"Subcategory: {s['name']}")
    s['soups'] = []
    s['soups'].append(soup_content_from_url(driver, s['url']))
    print(w['url'])
    time.sleep(5)

Subcategory: Accesorios
https://www.amazon.es/gp/bestsellers/computers/ref=zg_bs_nav_%i?ie=UTF8&pg=%i
Subcategory: Componentes
https://www.amazon.es/gp/bestsellers/computers/ref=zg_bs_nav_%i?ie=UTF8&pg=%i
Subcategory: Dispositivos de red
https://www.amazon.es/gp/bestsellers/computers/ref=zg_bs_nav_%i?ie=UTF8&pg=%i
Subcategory: Monitores
https://www.amazon.es/gp/bestsellers/computers/ref=zg_bs_nav_%i?ie=UTF8&pg=%i
Subcategory: Ordenadores de sobremesa
https://www.amazon.es/gp/bestsellers/computers/ref=zg_bs_nav_%i?ie=UTF8&pg=%i
Subcategory: Portátiles
https://www.amazon.es/gp/bestsellers/computers/ref=zg_bs_nav_%i?ie=UTF8&pg=%i
Subcategory: Servidores
https://www.amazon.es/gp/bestsellers/computers/ref=zg_bs_nav_%i?ie=UTF8&pg=%i
Subcategory: Tablets
https://www.amazon.es/gp/bestsellers/computers/ref=zg_bs_nav_%i?ie=UTF8&pg=%i
Subcategory: Almacenamiento de datos
https://www.amazon.es/gp/bestsellers/computers/ref=zg_bs_nav_%i?ie=UTF8&pg=%i
Subcategory: Barebones
https://www.amazon.es/gp/b

### Extracting data from html content subcategory
We need information about the products in that category like how much is a product sold(badge), the product name, the price and the url.

In [21]:
import locale
# $ sudo locale-gen es_ES
# $ sudo locale-gen es_ES.UTF-8
locale.setlocale(locale.LC_ALL, 'es_ES.UTF-8')

elements_df = []
elements_keys = []
for s in subcategories:
    edf = elements_from_soup_bestsellers_web(s['soups'][0])
    edf = edf.set_index('Badge')
    elements_df.append(edf)
    elements_keys.append(s['name'])
df = pd.concat(elements_df, keys=elements_keys)
df.Price = df.Price.str.replace('\xa0','').map(lambda x: locale.atof(x)).astype(float)
df.head(3)

Product  Price  \
           Badge                                                             
Accesorios #1     Amazon Fire TV Stick con mando por voz Alexa |...  39.99   
           #2     Logitech B100 Ratón con Cable, 3 Botones, Segu...   4.99   
           #3     Rampow Cable Lightning Cable Cargador iPhone-[...   8.99   

                                                               Link  
           Badge                                                     
Accesorios #1     /amazon-fire-tv-stick-con-mando-por-voz-alexa-...  
           #2     /Logitech-B100-Rat%C3%B3n-%C3%B3ptico-color/dp...  
           #3     /Rampow-Lightning-Cargador-iPhone-Certificado/...

In [22]:
df.to_csv(category['name']+'_subcategories.csv')

So, we have our data in csv format, yeah!. After I look our dataframe, I was thinking about if we had the sellers quantity offering each product, and the product id, we will find specific products in other websites. And where is that info? yep, it's in the product page, so we just need to navigate to each one, and add that information in our dataframe.

## Scrapping Amazon Product Pages
So, we'll iterate each product in our dataframe, getting its product page. This will be a slow process because we don't want a ban from amazon.

In [24]:
soups = {}
for i,r in df.iterrows():    
    soups[r.Link] = soup_content_from_url(driver, base_url+r.Link)
    time.sleep(random.randint(1,5))

In [25]:
len(soups)

615

### Saving pickle object
We do this because if we lost our data, we have a little backup and we don't need to make again requests to amazon.

In [26]:
save_obj(soups,'computer_soups')

Loading saved data (optional)

In [27]:
# soups = load_obj('product_soups')

### Getting detail from html content
We process the soup data in order to get the product details in pandas dataframe format

In [31]:
url="/Memoria-Flash-SanDisk-Velocidad-Lectura/dp/B077VYCV37/ref=zg_bs_17478027031_10?_encoding=UTF8&psc=1&refRID=FXVGHN6MZ1GYQXED23HA"
detail_dict_from_product_page(soups[url],url)

None
fail looking:/Memoria-Flash-SanDisk-Velocidad-Lectura/dp/B077VYCV37/ref=zg_bs_17478027031_10?_encoding=UTF8&psc=1&refRID=FXVGHN6MZ1GYQXED23HA


{}

In [32]:
soups[url]

<html class="a-no-js" lang="es"><!--<![endif]--><head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title dir="ltr">Amazon CAPTCHA</title>
<meta content="width=device-width" name="viewport"/>
<link href="https://images-na.ssl-images-amazon.com/images/G/01/AUIClients/AmazonUI-3c913031596ca78a3768f4e934b1cc02ce238101.secure.min._V1_.css" rel="stylesheet"/>
<script>

if (true === true) {
    var ue_t0 = (+ new Date()),
        ue_csm = window,
        ue = { t0: ue_t0, d: function() { return (+new Date() - ue_t0); } },
        ue_furl = "fls-eu.amazon.es",
        ue_mid = "A1RKKUPIHCS9HS",
        ue_sid = (document.cookie.match(/session-id=([0-9-]+)/) || [])[1],
        ue_sn = "opfcaptcha.amazon.es",
        ue_id = 'SHDEGMEMPRF4QPBMG6N2';
}
</script>
<script src="https://images-eu.ssl-images-amazon.com/images/G/01/csminstrumentation/csm-captcha-instrumentation.min.js"></scr

In [29]:
details = []
for url,soup in soups.items():
    d = detail_dict_from_product_page(soup,url)
    d['Link'] = url
    details.append(d)
detaildf = pd.DataFrame(details)
detaildf.head(2)

fail looking:/amazon-fire-tv-stick-con-mando-por-voz-alexa-reproductor-de-contenido-multimedia-en-streaming/dp/B07PVCVBN7/ref=zg_bs_937753031_1?_encoding=UTF8&psc=1&refRID=CPPZKFDMAXDJDGM7E55E
fail looking:/amazon-amazon-fire-tv-stick-4k-alexa-reproductor-de-contenido-multimedia-en-streaming/dp/B07PW9VBK5/ref=zg_bs_937753031_9?_encoding=UTF8&psc=1&refRID=CPPZKFDMAXDJDGM7E55E
fail looking:/Apple-MLA22Y-A-Teclado/dp/B016UPAXBE/ref=zg_bs_937912031_15?_encoding=UTF8&psc=1&refRID=3KKH8048XN9KR57XCD0E
fail looking:/Apple-iMac-Ordenador-Quad-Core-Teclado/dp/B071K2RVHF/ref=zg_bs_937994031_22?_encoding=UTF8&psc=1&refRID=2G1W036NNG69CMNR2AAX
fail looking:/Apple-Mac-mini-Ordenador-procesador/dp/B07K2RP84Q/ref=zg_bs_937994031_37?_encoding=UTF8&psc=1&refRID=2G1W036NNG69CMNR2AAX
fail looking:/Apple-MacBook-13-pulgadas-n%C3%BAcleo-generaci%C3%B3n/dp/B0863G2M7F/ref=zg_bs_938008031_5?_encoding=UTF8&psc=1&refRID=XN7VTQ2S7MEZN3VBQXEF
fail looking:/Nuevo-Apple-MacBook-pulgadas-almacenamiento/dp/B081G9YQ73

,Link,Marca,Series,Peso del producto,Dimensiones del producto,Número de modelo del producto,Color,Factor de forma,Fabricante del procesador,Tipo de procesador,...,Sistema de medida,Peso,Resolución horizontal,Potencia,Estabilización de imagen,Tamaño del folio,Volumen,Tipo de salida de la impresora,Idioma principal,Tipo(s) de material
0,/amazon-fire-tv-stick-con-mando-por-voz-alexa-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/Logitech-B100-Rat%C3%B3n-%C3%B3ptico-color/dp...,Logitech,B100,"99,8 g",13 x 9 x 5 cm,910-003357,Negro,Ambidextro,Intel,Ninguno,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# Avoiding merge conflicts with Link ref query param
df['link'] = df['Link'].map(lambda x: x.split('ref=')[0])
detaildf['link'] = detaildf['Link'].map(lambda x: x.split('ref=')[0])
# Merging
dfe = pd.merge(df.reset_index(),detaildf,on='link')
dfe = dfe.drop(['Link_x','Link_y'],axis=1)
dfe.head(3)

,level_0,Badge,Product,Price,link,Marca,Series,Peso del producto,Dimensiones del producto,Número de modelo del producto,...,Sistema de medida,Peso,Resolución horizontal,Potencia,Estabilización de imagen,Tamaño del folio,Volumen,Tipo de salida de la impresora,Idioma principal,Tipo(s) de material
0,Accesorios,#1,Amazon Fire TV Stick con mando por voz Alexa |...,39.99,/amazon-fire-tv-stick-con-mando-por-voz-alexa-...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Accesorios,#2,"Logitech B100 Ratón con Cable, 3 Botones, Segu...",4.99,/Logitech-B100-Rat%C3%B3n-%C3%B3ptico-color/dp...,Logitech,B100,"99,8 g",13 x 9 x 5 cm,910-003357,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Accesorios,#3,Rampow Cable Lightning Cable Cargador iPhone-[...,8.99,/Rampow-Lightning-Cargador-iPhone-Certificado/...,RAMPOW,NaN,"18,1 g","22,8 x 20,8 x 4 cm",RAMPOW01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
list(dfe.columns)

['level_0',
 'Badge',
 'Product',
 'Price',
 'link',
 'Marca',
 'Series',
 'Peso del producto',
 'Dimensiones del producto',
 'Número de modelo del producto',
 'Color',
 'Factor de forma',
 'Fabricante del procesador',
 'Tipo de procesador',
 'Tipo de memoria del ordenador',
 'Interfaz del disco duro',
 'Voltaje',
 'Fuente de alimentación',
 'Plataforma de Hardware',
 'Sistema operativo',
 'ASIN',
 'Valoración media de los clientes',
 'Clasificación en los más vendidos de Amazon',
 'Producto en Amazon.es desde',
 'sellers',
 'Modelo',
 'Nombre del modelo',
 'Número de producto',
 'Aparatos compatibles',
 'Otras características',
 'Número de productos',
 'Pantalla a color',
 'Pilas / baterías incluidas',
 'Pilas / baterías necesarias',
 'Enfoque automático',
 'Incluye mando',
 'Programable',
 'Restricciones de envío',
 'Porcentaje total de reciclaje de contenido',
 'Tamaño',
 'Color de la tinta',
 'Número de pieza del fabricante',
 'Pilas:',
 'Dimensión de la pantalla',
 'Número de proc

In [35]:
dffinal = dfe[['level_0','Badge','Product','Price','link','Marca','Modelo','Color','Número de productos','Tamaño','Número de pieza del fabricante','ASIN','sellers','Valoración media de los clientes','Clasificación en los más vendidos de Amazon','Número de modelo del producto']]

In [36]:
dffinal.to_csv('computers_subcategories_extended.csv', index=False)

In [37]:
df.sum()

Product    Amazon Fire TV Stick con mando por voz Alexa |...
Price                                                88535.1
Link       /amazon-fire-tv-stick-con-mando-por-voz-alexa-...
link       /amazon-fire-tv-stick-con-mando-por-voz-alexa-...
dtype: object

In [38]:
len(dffinal)

633

In [142]:
import pandas as pd
df=pd.DataFrame({"x":[1,2,3,4,5],"y":[2,4,6,8,10]})
df.sum(axis=0)

x    15
y    30
dtype: int64

In [143]:
df

,x,y
0,1,2
1,2,4
2,3,6
3,4,8
4,5,10


In [146]:
df.sum(axis=0)

x    15
y    30
dtype: int64